# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

#Print a "All clear"
print("Good to go!")

Good to go!


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Create the Weather Dataframe
weather_df = pd.read_csv('output_data/cities.csv')

#Display DataFrame
weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure Gmaps
gmaps.configure(api_key=g_key)

#Define data
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [9]:
#Plot, define type of map
fig = gmaps.figure(map_type = 'HYBRID')

#Add the Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity = 8, point_radius = 1)
fig.add_layer(heat_layer)

#Display gmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Created a narrowed DataFrame
narrow_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]

#Display narrowed DataFrame
narrow_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#Create a hotel DataFrame
hotel_df = narrow_df.copy()

#Create a "Hotel Name" column
hotel_df["Hotel Name"] = ""

#Display DataFrame
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,


In [17]:
#Create a dictionary to update with each cycle
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword":"hotel",
    "key": g_key
}

#Add lat and long values as it goes over dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    hotel_address = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Name"] = hotel_address["results"][0]["name"]
        hotel_df.loc[index,"Address"] = hotel_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing hotel: {} So I will skip that." .format(index))

#Save to output file
output_file = "Output_Data/Hotels.csv"
hotel_df.to_csv(output_file)

#Display Hotel Dataframe
hotel_df.head(15)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Name,Address
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,,Opuwo Country Lodge,Opuwo
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,,Hotel Capitólio,"R. Passos Maia, 121 - Centro, Capitólio"
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,,Pond Country Club,"Estrada RS 784 KM 7 - Lagoa Country Club, Cidr..."
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,,Kimony Resort Hotel,Morondava
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,,Dandeli Mist Jungle Stay,"12km from Ramnagar, Chandwadi road, Shinoli, J..."
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,,"Hotel Marchica Lagoon Resort, Nador Morocco",Cité d'Atalayoun
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,,Mount Mogok Hotel,Mogok
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,,Sepehr Hotel,Birjand


In [22]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

#Display map
fig

Figure(layout=FigureLayout(height='420px'))